In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
import warnings
warnings.filterwarnings('ignore')

import gym
import multiprocessing
import threading
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## Creating the mountain car environment

Let's create a mountain car environment using the gym. Note that our mountain car
environment is a continuous environment meaning that our action space is continuous:

In [4]:
env = gym.make('MountainCarContinuous-v0')

In [5]:
state_shape = env.observation_space.shape[0]
state_shape

2

In [6]:
action_shape = env.action_space.shape[0]
action_shape

1

In [7]:
action_bound = [env.action_space.low, env.action_space.high]
action_bound

[array([-1.], dtype=float32), array([1.], dtype=float32)]

## Defining the variables


Now, let's define some of the important variables.

Define the number of workers as the number of CPUs:

In [8]:
num_workers = multiprocessing.cpu_count() 
num_workers

4

In [ ]:
num_episodes = 2000 
num_timesteps = 200 

In [14]:
# Global network ( global agent) scopte
global_net_scope = 'Global_Net'

# The time step at which we want to update the global network
update_global = 10

# discount factr 
gamma = 0.9

# beta
beta = 0.01

# The directory where we want to store the logs
log_dir = "logs"

## Defining the actor critic class


In [14]:
def build_network(scope):
    return 10, 5, 4, 1, 1

In [15]:
scope = "o"
global_net_scope = "o"

actor_optimizer = tf.train.RMSPropOptimizer(0.0001, name="RMSPropA")
critic_optimizer = tf.train.RMSPropOptimizer(0.0001, name="RMSPropC")


if scope == global_net_scope:
    with tf.variable_scope(scope):
        state = tf.placeholder(tf.float32, [None, state_shape])
        actor_params, critic_params = build_network(scope)[-2:]
        
else:
    with tf.variable_scope(scope):
        state = tf.placeholder(tf.float32, [None, state_shape], "state")
        action_dist = tf.placeholder(tf.float32, [None, action_shape], "action")
        target_value = tf.placeholder(tf.float32, [None, 1], "Vtarget")
        
        mean, variance, value, actor_params, critic_params = build_network(scope)
        
        td_error = tf.subtract(target_value, value, name="TD_error")
        
        with tf.name_scope("critic_loss"):
            critic_loss = tf.reduce_mean(tf.square(td_error))
        
        with tf.name_scope("wrap_action"):
            mean, variance = mean * action_bound[1], variance + 1e-4
        
        normal_dist = tf.distributions.Normal(mean, variance)
        
        with tf.name_scope("actor_loss"):
            log_prob = normal_dist.log_prob(action_dist)
            entropy_pi = normal_dist.entropy()
            
            loss = log_prob * td_error + (beta * entropy_pi)
            action_loss = tf.reduce_mean(-loss)
        
        with tf.name_scope("select_action"):
            action = tf.clip_by_value(tf.squeeze(normal_dist.sample(1), axis=0),
                                      action_bound[0], action_bound[1])
        
        with tf.name_scope("local_grad"):
            actor_grads = tf.gradients(actor_loss, actor_params)
            critic_grads = tf.gradients(critic_loss, critic_params)
            
    with tf.name_scope("sync"):
        
        with tf.name_scope("push"):
            update_actor_params = actor_optimizer.apply_gradients(zip(actor_grads,
                                                                      globalAC.actor_params))
            update_critic_params = critic_optimizer.apply_gradients(zip(actor_grads,
                                                                        globalAC.critic_params))
            
        with tf.name_scope("pull"):
            pull_actor_params = [l_p.assign(g_p) for l_p, g_p in zip(actor_params,
                                                                     globalAC.actor_params)]
            pull_critic_params = [l_p.assign(g_p) for l_p, g_p in zip(critic_params,
                                                                      globalAC.critic_params)]
            

In [16]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph)
    print(sess.run(sum))

TypeError: Argument `fetch` = <built-in function sum> has invalid type "builtin_function_or_method" must be a string or Tensor. (Can not convert a builtin_function_or_method into a Tensor or Operation.)